In [5]:
! wget --header="Host: doc-0k-80-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9,so;q=0.8" --header="Cookie: AUTH_em2qh84qf50cu7hgm8cc1jk92s9uldlr_nonce=srer3d63fautk" --header="Connection: keep-alive" "https://doc-0k-80-docs.googleusercontent.com/docs/securesc/e2th8t97ida4c7qm0bld5ql54cdmkla7/1njv9622c25go0qn7us7o14ejgtjfph1/1608216300000/06629147635963609455/08403420218720776473/1Lz1IHzyhCoWTsDuaD9Kf7OyoLK4XdFTr?e=download&authuser=0&nonce=srer3d63fautk&user=08403420218720776473&hash=kljfh203r5jtrindcmknadce3u1jvirs" -c -O 'preprocessed_data.csv'

--2020-12-17 14:45:43--  https://doc-0k-80-docs.googleusercontent.com/docs/securesc/e2th8t97ida4c7qm0bld5ql54cdmkla7/1njv9622c25go0qn7us7o14ejgtjfph1/1608216300000/06629147635963609455/08403420218720776473/1Lz1IHzyhCoWTsDuaD9Kf7OyoLK4XdFTr?e=download&authuser=0&nonce=srer3d63fautk&user=08403420218720776473&hash=kljfh203r5jtrindcmknadce3u1jvirs
Resolving doc-0k-80-docs.googleusercontent.com (doc-0k-80-docs.googleusercontent.com)... 173.194.79.132, 2a00:1450:4013:c05::84
Connecting to doc-0k-80-docs.googleusercontent.com (doc-0k-80-docs.googleusercontent.com)|173.194.79.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘preprocessed_data.csv’

preprocessed_data.c     [        <=>         ] 118.69M  50.8MB/s    in 2.3s    

2020-12-17 14:45:46 (50.8 MB/s) - ‘preprocessed_data.csv’ saved [124454659]



In [6]:
! wget --header="Host: doc-0o-80-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9,so;q=0.8" --header="Cookie: AUTH_em2qh84qf50cu7hgm8cc1jk92s9uldlr=08403420218720776473|1608216300000|5qfrp4if5mt2pg85lapq14b6vim1a4m8" --header="Connection: keep-alive" "https://doc-0o-80-docs.googleusercontent.com/docs/securesc/e2th8t97ida4c7qm0bld5ql54cdmkla7/fq85kkgh0iqtpsgirva5il8r4ht63rif/1608216300000/06629147635963609455/08403420218720776473/1Z6bjXmyCaoEzXYo_tRDwLTsfeA2F3K3j?e=download&authuser=0" -c -O 'glove_vectors'

--2020-12-17 14:46:16--  https://doc-0o-80-docs.googleusercontent.com/docs/securesc/e2th8t97ida4c7qm0bld5ql54cdmkla7/fq85kkgh0iqtpsgirva5il8r4ht63rif/1608216300000/06629147635963609455/08403420218720776473/1Z6bjXmyCaoEzXYo_tRDwLTsfeA2F3K3j?e=download&authuser=0
Resolving doc-0o-80-docs.googleusercontent.com (doc-0o-80-docs.googleusercontent.com)... 173.194.79.132, 2a00:1450:4013:c05::84
Connecting to doc-0o-80-docs.googleusercontent.com (doc-0o-80-docs.googleusercontent.com)|173.194.79.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘glove_vectors’

glove_vectors           [     <=>            ] 121.60M   127MB/s    in 1.0s    

2020-12-17 14:46:18 (127 MB/s) - ‘glove_vectors’ saved [127506004]



In [711]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras 
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.initializers import RandomUniform,he_uniform
from keras import optimizers
from keras.callbacks import Callback

In [712]:
dataset = pd.read_csv('preprocessed_data.csv')
dataset.columns

Index(['school_state', 'teacher_prefix', 'project_grade_category',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'clean_categories', 'clean_subcategories', 'essay', 'price'],
      dtype='object')

In [713]:
y= dataset["project_is_approved"]
dataset.drop("project_is_approved",axis = 1, inplace = True)
X=dataset
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.20, stratify=y)

In [714]:
from sklearn.metrics import roc_auc_score

class Metrics(Callback):    
  def __init__(self,trainingdata,validationdata,batch_size = 128):

    super(Metrics,self).__init__()
    self.x = trainingdata[0]
    self.y = trainingdata[1]
    self.x_test = validationdata[0]
    self.y_test = validationdata[1]
    
  def on_train_begin(self, logs={}):
    self.val_auc = []

  def on_epoch_end(self, epoch, logs={}):

    val_predict = np.array(self.model.predict(self.x_test)).round()
    val_pred = np.array(self.model.predict(self.x_test))
    val_targ = self.y_test
    _val_auc =roc_auc_score(val_targ, val_pred)
    self.val_auc.append(_val_auc)
    print("Auc: %f " %(_val_auc))
    return    

validation_data = (X_cv,y_cv)
training_data = (X_train,y_train) 
metrics = Metrics(training_data,validation_data)

In [715]:
def auc1(y_true, y_pred):
    if len(np.unique(y_true[:,1])) == 1:
        return 0.5
    else:
        return roc_auc_score(y_true, y_pred)

def auroc(y_true, y_pred):

  return tf.py_function(auc1, (y_true, y_pred), tf.double)

Encoding Essay

In [716]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['essay'])

In [717]:
X_train['essay_tokens'] = tokenizer.texts_to_sequences(X_train['essay'].values)
X_cv['essay_tokens'] = tokenizer.texts_to_sequences(X_cv['essay'].values)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [718]:
from keras.preprocessing import sequence
max_review_length = 300
X_train_pad = sequence.pad_sequences(X_train['essay_tokens'].values, maxlen=max_review_length , padding='post' )

X_cv_pad  = sequence.pad_sequences(X_cv['essay_tokens'].values, maxlen=max_review_length , padding='post')


In [719]:
import pickle
with open('glove_vectors', 'rb') as f:
    glove = pickle.load(f)
    glove_words =  set(glove.keys())

In [720]:
from numpy import zeros
max_vocabulary = len(tokenizer.word_index)
embedding_matrix = zeros((max_vocabulary+1, 300))
for word, i in tokenizer.word_index.items():
  if word in glove_words:
    embedding_vector = glove[word]
    embedding_matrix[i] = embedding_vector

Encoding Numerical Features

In [721]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()

normalizer.fit(X_train['teacher_number_of_previously_posted_projects'].values.reshape(1,-1))

X_train_number_norm = normalizer.transform(X_train['teacher_number_of_previously_posted_projects'].values.reshape(1,-1))
X_cv_number_norm = normalizer.transform(X_cv['teacher_number_of_previously_posted_projects'].values.reshape(1,-1))

X_train_number_norm = X_train_number_norm.reshape(-1,1)
X_cv_number_norm = X_cv_number_norm.reshape(-1,1)


print("After vectorizations")
print(X_train_number_norm.shape, y_train.shape)
print(X_cv_number_norm.shape, y_cv.shape)
print("="*100)



After vectorizations
(87398, 1) (87398,)
(21850, 1) (21850,)


In [722]:
normalizer.fit(X_train['price'].values.reshape(1,-1))

X_train_price_norm = normalizer.transform(X_train['price'].values.reshape(1,-1))
X_cv_price_norm = normalizer.transform(X_cv['price'].values.reshape(1,-1))

X_train_price_norm = X_train_price_norm.reshape(-1,1)
X_cv_price_norm = X_cv_price_norm.reshape(-1,1)

print("After vectorizations")
print(X_train_price_norm.shape, y_train.shape)
print(X_cv_price_norm.shape, y_cv.shape)
print("="*100)

After vectorizations
(87398, 1) (87398,)
(21850, 1) (21850,)


In [723]:
X_train_numeric = np.concatenate([X_train_price_norm,X_train_number_norm],axis=1)
X_cv_numeric = np.concatenate([X_cv_price_norm,X_cv_number_norm],axis=1)
print(X_train_numeric.shape,X_cv_numeric.shape)

(87398, 2) (21850, 2)


Encoding Categorical Features

In [724]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()

encode.fit(X_train['school_state'].values)
X_train_school_state = encode.transform(X_train['school_state'].values)
X_cv_school_state = encode.transform(X_cv['school_state'].values)
X_train_school_state_unique = np.unique(X_train_school_state)

encoded_size_school = len(X_train_school_state_unique) - len(X_train_school_state_unique)/2
encoded_size_school = np.ceil(encoded_size_school)
encoded_size_school = int(encoded_size_school)
print(encoded_size_school)
X_train_school_state_unique = len(X_train_school_state_unique)
print(X_train_school_state_unique)


26
51


In [725]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()

encode.fit(X_train['teacher_prefix'].values)
X_train_teacher = encode.transform(X_train['teacher_prefix'].values)
X_cv_teacher = encode.transform(X_cv['teacher_prefix'].values)
X_train_teacher_unique = np.unique(X_train_teacher)

encoded_teacher = len(X_train_teacher_unique) - len(X_train_teacher_unique)/2
encoded_teacher = int(np.ceil(encoded_teacher))
print(encoded_teacher)
X_train_teacher_unique = len(X_train_teacher_unique)


3


In [726]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()

encode.fit(X_train['project_grade_category'].values)
X_train_project = encode.transform(X_train['project_grade_category'].values)
X_cv_project = encode.transform(X_cv['project_grade_category'].values)
X_train_project_unique = np.unique(X_train_project)

encoded_project = len(X_train_project_unique) - len(X_train_project_unique)/2
encoded_project = int(np.ceil(encoded_project))
print(encoded_project)
X_train_project_unique = len(X_train_project_unique)


2


In [727]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()

encode.fit(X_train['clean_categories'].values)
X_train_clean = encode.transform(X_train['clean_categories'].values)

X_cv_clean = X_cv['clean_categories'].map(lambda s: '<unknown>' if s not in encode.classes_ else s)
encode.classes_ = np.append(encode.classes_, '<unknown>')
X_cv_clean= encode.transform(X_cv_clean)

X_cv_clean = encode.transform(X_cv['clean_categories'].values)
X_train_clean_unique = np.unique(X_train_clean)

encoded_clean = len(X_train_clean_unique) - len(X_train_clean_unique)/2
encoded_clean = int(np.ceil(encoded_clean))
print(encoded_clean)
X_train_clean_unique = len(X_train_clean_unique)

26


In [728]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()

encode.fit(X_train['clean_subcategories'])
X_train_clean_sub = encode.transform(X_train['clean_subcategories'].values)

X_cv_clean_sub = X_cv['clean_subcategories'].map(lambda s: '<unknown>' if s not in encode.classes_ else s)
encode.classes_ = np.append(encode.classes_, '<unknown>')
X_cv_clean_sub = encode.transform(X_cv_clean_sub)

X_train_clean_sub_unique = np.unique(X_train_clean_sub)


encoded_clean_sub = len(X_train_clean_sub_unique) - len(X_train_clean_sub_unique)/2
encoded_clean_sub = int(np.ceil(encoded_clean_sub))
print(encoded_clean_sub)
X_train_clean_sub_unique = len(X_train_clean_sub_unique)

198


Model LSTM

In [729]:
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras import regularizers
from keras.regularizers import l2
from keras.layers import Flatten
from keras.layers import Dense, Input , Dropout
from keras.layers import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard

In [730]:
input = Input(shape=(300,))
X = Embedding(output_dim=300, input_dim = max_vocabulary+1, input_length=300 , weights=[embedding_matrix])(input)
lstm_1 = LSTM(90, activation='relu',return_sequences=True)(X)
flatten_1 = Flatten()(lstm_1)

school = Input(shape=(1,))
X_school = Embedding(output_dim = encoded_size_school, input_dim= X_train_school_state_unique+1, input_length=1)(school)
flatten_2 = Flatten()(X_school)

teacher = Input(shape=(1,))
X_teacher = Embedding(output_dim= encoded_teacher , input_dim= X_train_teacher_unique+1, input_length=1)(teacher)
flatten_3 = Flatten()(X_teacher)

clean = Input(shape=(1,))
X_clean = Embedding(output_dim= encoded_clean, input_dim= X_train_clean_unique+1, input_length=1)(clean)
flatten_4 = Flatten()(X_clean)

clean_sub = Input(shape=(1,))
X_clean_sub = Embedding(output_dim= encoded_clean_sub, input_dim= X_train_clean_sub_unique+1, input_length=1)(clean_sub)
flatten_5 = Flatten()(X_clean_sub)

project = Input(shape=(1,))
X_project = Embedding(output_dim= encoded_project, input_dim = X_train_project_unique+1,input_length=1)(project)
flatten_6 = Flatten()(X_project)

X_numbers = Input(shape=(2,))
numeric = Dense(482, activation='relu',kernel_initializer='glorot_normal')(X_numbers)

X_concatenate = concatenate([flatten_1 , flatten_2 , flatten_3 ,flatten_4 , flatten_5 , flatten_6 , numeric])

model = Dense(200, activation="relu", kernel_initializer="glorot_normal")(X_concatenate)
#model = BatchNormalization()(model)
model = Dropout(0.4)(model)


model = Dense(100,activation="relu",kernel_initializer="glorot_normal")(model)
model = Dropout(0.4)(model)

model = Dense(42, activation="relu", kernel_initializer="glorot_uniform")(model)
#model = BatchNormalization()(model)

model = Dropout(0.4)(model)


output = Dense(2, activation='sigmoid', name='output')(model)

model_ready = Model(inputs = [input,school,teacher,clean,clean_sub,project,X_numbers], outputs = output)
print(model_ready.summary())


Model: "functional_48"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_224 (InputLayer)          [(None, 300)]        0                                            
__________________________________________________________________________________________________
embedding_186 (Embedding)       (None, 300, 300)     15544200    input_224[0][0]                  
__________________________________________________________________________________________________
input_225 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_226 (InputLayer)          [(None, 1)]          0                                            
______________________________________________________________________________________

In [731]:
train_all = [X_train_pad,X_train_school_state,X_train_teacher,X_train_clean,X_train_clean_sub,X_train_project,X_train_numeric]

test_all = [X_cv_pad,X_cv_school_state,X_cv_teacher,X_cv_clean, X_cv_clean_sub, X_cv_project, X_cv_numeric]

In [732]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 2) 
y_cv = np_utils.to_categorical(y_cv, 2)

In [733]:
def decrease(epoch, lr):    
  if (epoch%3 == 0 and epoch!=0):    
    lr = lr - (0.1 * lr)
    return lr
  else:
    return lr

In [734]:
DynamicLR = ReduceLROnPlateau(monitor='val_auroc', factor=0.9, patience=1, verbose=1, mode='max',min_delta=0, cooldown=0, min_lr=0.0001)
adaptiveLR =  LearningRateScheduler(decrease)
adam = tf.keras.optimizers.Adam(0.001)
callbacks_1 = [DynamicLR,adaptiveLR]

In [735]:
model_ready.compile(optimizer=adam, loss='categorical_crossentropy', metrics=[auroc])

In [736]:
history = model_ready.fit(train_all, y_train, batch_size=256, epochs=3, verbose=1,callbacks=callbacks_1, validation_data=(test_all, y_cv))

Epoch 1/3
342/342 [==============================] - 203s 594ms/step - loss: 0.4060 - auroc: 0.6845 - val_loss: 0.3748 - val_auroc: 0.7446
Epoch 2/3
342/342 [==============================] - 203s 595ms/step - loss: 0.3646 - auroc: 0.7672 - val_loss: 0.3714 - val_auroc: 0.7471
Epoch 3/3
342/342 [==============================] - ETA: 0s - loss: 0.3190 - auroc: 0.8370
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0009000000427477062.
342/342 [==============================] - 203s 594ms/step - loss: 0.3190 - auroc: 0.8370 - val_loss: 0.3921 - val_auroc: 0.7278


In [2]:
table = pandas.DataFrame({
    "Vectorizer":['Word Embedding'],
    "Model":["LSTM"],
    "Trainable Parameters":[21339574],
    "AUC":[0.728],
    
})
table

,Vectorizer,Model,Trainable Parameters,AUC
0,Word Embedding,LSTM,21339574,0.728
